### Modelos de Recuperaciòn de Informaciòn (y evaluaciòn)

Utilizando la colecci´on provista por el equipo docente1
, cuya estructura es la siguiente:
    
vocabulary.txt → [id termino, idf, t´ermino]

documentVectors.txt → [id doc, lista(id terminos)]

queries.txt → [id query, lista(id terminos)]

relevants.txt → [id query, listarelevantes (id doc)]

informationNeeds.txt → [id in, texto libre]


a) Calcule los conjuntos de respuestas usando el modelo booleano y el modelo vectorial (asuma en todos
los casos T F = 1).

In [136]:
from os import listdir
from os.path import isdir
import math
from modulos.tokenizer import *

In [137]:
def calc_idf(corpus_count,doc_freq):
    if(doc_freq !=0):
        return math.log(corpus_count/doc_freq,2)
    return 0

In [138]:
def indexer(dirname):
        files = listdir(dirname)
        vocabulary={}
        document_vector={}
        id_voc=0
        vocavulary_result=[]
        docs_count=len(files)
        for file in files:
            lines = open(dirname+'/'+file,'r',errors = 'ignore').readlines()
            docu_voc=[]
            tokens=tokenizar(lines)
            #docs_count+=1
            for token in tokens:
                if token not in vocabulary:
                    vocabulary[token]=(id_voc,1)
                    docu_voc.append(id_voc)
                    id_voc =id_voc+1
                    #print(token,docu_voc)
                    
                    
                else:
                    id,doc_freq = vocabulary[token]
                    
                    #print(id,docu_voc)
                    if id not in docu_voc:
                       
                        doc_freq = doc_freq+1
                        docu_voc.append(id)
                        
                    vocabulary[token]=(id,doc_freq)
                    
            document_vector[file]=docu_voc
        #print(document_vector)
        #print(vocabulary)
        for termn in vocabulary.items():
            #print(termn[1][1],docs_count)
            vocavulary_result.append([termn[1][0],calc_idf(docs_count,termn[1][1]),termn[0]])
        #print (vocavulary_result)
        return (vocavulary_result,document_vector)

In [166]:
vocabulary,documents=  indexer('/home/agu/Unlu/IR/coleccion_2018_final/')
#print(vocabulary)
#print(documents)

In [167]:
import numpy as np
from operator import itemgetter

In [303]:
#voc = (list(get_vocabulary("ejemploRibeiro/vocabulary.txt", True)))
#doc = (get_document_vector("ejemploRibeiro/documentVector.txt", True))
voc = vocabulary
vocabulary_list=list(map(itemgetter(2), voc))
#doc = documents
#doc_bool= to_boolean(doc,voc)
print (voc[1:20])
#print(documents)

[[1, 3.1043366598147353, 'ingenios'], [2, 3.1043366598147353, 'coleccion'], [3, 0.0, 'de'], [4, 2.2563397532597858, 'vistas'], [5, 0.9505313237357003, 'principales'], [6, 4.256339753259786, 'azucar'], [7, 0.0, 'la'], [8, 1.0339473319233377, 'isla'], [9, 2.2563397532597858, 'cuba'], [10, 2.7258250365610057, 'edicion'], [11, 1.3818706353436445, 'lujo'], [12, 0.0, 'el'], [13, 1.8115549105868898, 'texto'], [14, 4.104336659814735, 'redactado'], [15, 0.0, 'por'], [16, 0.6580804299251717, 'justo'], [17, 5.841302253980943, 'cantero'], [18, 1.6189098326444942, 'gentil'], [19, 0.19744606420621716, 'hombre']]


In [273]:
def get_query(query,vocabulary_list):
    
    query_result=[]
    querys = query.split()
    query_terms=[]
    terms_count={}
    query_freq=[]
    for term  in querys:
        if term not in query_terms:
            query_terms.append(term)
            terms_count[term]=1
            if term in vocabulary_list:
                query_result.append(vocabulary_list.index(term))  
        else:
            terms_count[term]+=1
    for term,count in terms_count.items():
        query_freq.append(count)
    return query_result, query_freq

In [343]:
query,query_freq =get_query("hello agustin dog dog dog",vocabulary_list)
print(query)
query_w =[]
print (query_freq)

[101491, 5215, 77383]
[1, 1, 3]


In [344]:
result=[]
query_documents=[]
for term in query:

    #print(term)
    
    for document, values in documents.items():
        #print(values)
        if term in values:
            if document not in query_documents:   
                query_documents.append(document)

print(query_documents)


['42323-8.txt', '39312-8.txt', '14307-8.txt', '44120-8.txt', '26028-8.txt', '18723-8.txt', '45151-8.txt', '27757-8.txt', '15421-8.txt', '27167-8.txt', '41884-8.txt', '19643-8.txt', '13216-8.txt', '11669-8.txt', '15633-8.txt', '13458-8.txt', '28904-8.txt', '45180-8.txt', '28967-8.txt', '20401-8.txt', '19106-8.txt', '30053-8.txt', '40358-8.txt']


In [345]:
def get_normal(list_term,vocabulary):
    query_normal=0
    for term in list_term:
        val=(vocabulary[term][1])
        query_normal+=float(val)**2
    query_normal=math.sqrt(query_normal)
    #print(query_normal,"fnc")
    return(query_normal)
def get_query_w(index, query_w,idf):
    #print(0,5*(query_w[index]/max(query_w))*idf,"qqqqqq")
    return 0.5+ 0.5*(query_w[index]/max(query_w))*idf
    

In [346]:
result=[]
print(query)
#print(get_normal(query,voc))
query_normal=get_normal(query,voc)

result_q=[]

terms_isin_documents={}

for term in query:
    if voc[term][1] > 0:
        query_document=[]
        for document, values in documents.items():
            #docu, value = doc
            if term in values:
                if (document not in [ids for ids, _ in result_q]):
                    query_document.append((document,voc[term][1]))

            if(term in terms_isin_documents.keys()):
                terms_isin_documents[str(term)].extend(query_document)     
            else:
                if len(query_document)>0:
                    terms_isin_documents[str(term)]=query_document
        result_q.extend(query_document)
#print("query",query_document) 
result.append(("query",result_q))

print("Boolean Result")
for doc,_ in result_q:
    print(doc)
#print(result_q)
query_vector=[]
for index,(key,value) in enumerate(terms_isin_documents.items()):

    idf=float(voc[int(key)][1])
    tf_maxtf= get_query_w(index,query_freq,float(voc[int(key)][1]))
    #query_vector.append(float(voc[int(key)][1]))
    #print(tf_maxtf)
    query_vector.append(tf_maxtf)


vector_result=[]
#print(query_vector,"qv")
for name,idf in result_q:
    #print(result_q)
    vect_sim =[]
    sim=0
    #value son los documentos que contienen los terminos
    #print(terms_isin_documents.items())
    for key,value in terms_isin_documents.items():
        if (name in [val[0] for val in value]):
            vect_sim.append(float(idf))
        else:
            vect_sim.append(0)
    vector_result.append((name,vect_sim))
print("------------------------")
print("Vectorial Result")
#print(vector_result)
list_response=[]
#print(query_normal)
for doc,vec in vector_result:
    #r=np.array(r)
    q=0

    for idf in vec:
        q+=idf**2
    q = math.sqrt(q)
    q = (np.dot(vec,query_vector)) /(query_normal*q)
    list_response.append((doc,q))


(list_response.sort(key=itemgetter(1),reverse=True))

for doc in list_response:
    print(doc)
print("-----------------")
    
    

[101491, 5215, 77383]
Boolean Result
42323-8.txt
39312-8.txt
14307-8.txt
44120-8.txt
26028-8.txt
18723-8.txt
45151-8.txt
27757-8.txt
15421-8.txt
27167-8.txt
41884-8.txt
19643-8.txt
13216-8.txt
11669-8.txt
15633-8.txt
13458-8.txt
28904-8.txt
45180-8.txt
28967-8.txt
20401-8.txt
19106-8.txt
30053-8.txt
40358-8.txt
------------------------
Vectorial Result
('30053-8.txt', 0.36050985485287856)
('40358-8.txt', 0.36050985485287856)
('42323-8.txt', 0.16871519835483872)
('39312-8.txt', 0.09877884310590605)
('14307-8.txt', 0.09877884310590605)
('44120-8.txt', 0.09877884310590605)
('26028-8.txt', 0.09877884310590605)
('18723-8.txt', 0.09877884310590605)
('45151-8.txt', 0.09877884310590605)
('27757-8.txt', 0.09877884310590605)
('15421-8.txt', 0.09877884310590605)
('27167-8.txt', 0.09877884310590605)
('41884-8.txt', 0.09877884310590605)
('19643-8.txt', 0.09877884310590605)
('13216-8.txt', 0.09877884310590605)
('11669-8.txt', 0.09877884310590605)
('15633-8.txt', 0.09877884310590605)
('13458-8.txt', 